In [2]:
import pandas as pd
# !pip install gensim
import re

In [3]:
news = pd.read_csv('dutch-news-articles.csv')

In [4]:
news.head()

,datetime,title,content,category,url
0,2010-01-01 00:49:00,Enige Litouwse kerncentrale dicht,De enige kerncentrale van Litouwen is oudjaars...,Buitenland,https://nos.nl/artikel/126231-enige-litouwse-k...
1,2010-01-01 02:08:00,Spanje eerste EU-voorzitter onder nieuw verdrag,Spanje is met ingang van vandaag voorzitter va...,Buitenland,https://nos.nl/artikel/126230-spanje-eerste-eu...
2,2010-01-01 02:09:00,Fout justitie in Blackwater-zaak,Vijf werknemers van het omstreden Amerikaanse ...,Buitenland,https://nos.nl/artikel/126233-fout-justitie-in...
3,2010-01-01 05:14:00,"Museumplein vol, minder druk in Rotterdam",Het Oud en Nieuwfeest op het Museumplein in Am...,Binnenland,https://nos.nl/artikel/126232-museumplein-vol-...
4,2010-01-01 05:30:00,Obama krijgt rapporten over aanslag,President Obama heeft de eerste rapporten gekr...,Buitenland,https://nos.nl/artikel/126236-obama-krijgt-rap...


In [5]:
regional_labels = ['L1mburg', 
                   'NH Nieuws', 
                   'Omroep Brabant', 
                   'Omrop Frylân', 
                   'Omroep Gelderland', 
                   'Omroep West',
                   'Omroep Zeeland',
                   'Rijnmond',
                   'RTV Rijnmond',
                   'RTV Drenthe',
                   'RTV Noord',
                   'RTV Oost',
                   'RTV Utrecht'
                  ]

def contains_label(text):
    return [label for label in regional_labels if label in text]

In [6]:
news['omroep'] = news.content.apply(contains_label)

In [7]:
news[news.omroep.apply(len)>0].shape

(15549, 6)

In [8]:
# news[(news['omroep'].apply(len)>1) & (news.content.str.contains('drug'))].reset_index().loc[0]

## Word2Vec

### Process text

In [9]:
def split_into_sentences(text):
    sentences = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])|\n', text)
    sentences = [x for x in sentences if len(x) > 1]
    return sentences

In [10]:
# remove punctuation

sentence_list_by_word = []
sentence_list = []

for i in range(len(news)):
    doc = news.iloc[i]['content']
    sentences = split_into_sentences(doc)
    sentence_list.append(sentences)
    for j in sentences:
        word_list = [x for x in j.lower().rstrip().replace('.', '').split(' ') if len(x)>0]
        sentence_list_by_word.append(word_list)
        


wordlen = 0
for i in sentence_list_by_word:
    wordlen += len(i)
    
print(f"Sentence count: {len(sentence_list_by_word)}")
print(f"Word count: {wordlen}")

Sentence count: 3477825
Word count: 62407090


### Train model

In [11]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

news_model = Word2Vec(sentences=sentence_list_by_word, vector_size=100, window=5, min_count=1, workers=4)
news_model.save("news_word2vec.model")
news_model = Word2Vec.load("news_word2vec.model")

In [14]:
# list_of_drugs = ['xtc', 'mdma', 'cocaine', 'wiet']
list_of_drugs = ['mdma']

top_n=10

sims={drug:news_model.wv.most_similar(drug, topn=top_n) for drug in list_of_drugs}

In [15]:
sims

{'mdma': [('amfetamine', 0.8705504536628723),
  ('methamfetamine', 0.8429946303367615),
  ('ketamine', 0.840532124042511),
  ('xtc', 0.8359721302986145),
  ('mdma,', 0.8297138214111328),
  ('cocaïne,', 0.822767972946167),
  ('partydrug', 0.8208773732185364),
  ('xtc,', 0.8182334899902344),
  ('fentanyl', 0.8048309087753296),
  ('meth', 0.8022823929786682)]}

In [16]:
pd.DataFrame(sims).T

,0,1,2,3,4,5,6,7,8,9
mdma,"(amfetamine, 0.8705504536628723)","(methamfetamine, 0.8429946303367615)","(ketamine, 0.840532124042511)","(xtc, 0.8359721302986145)","(mdma,, 0.8297138214111328)","(cocaïne,, 0.822767972946167)","(partydrug, 0.8208773732185364)","(xtc,, 0.8182334899902344)","(fentanyl, 0.8048309087753296)","(meth, 0.8022823929786682)"
